In [1]:
import torch
import numpy as np
import open3d as o3d
import gpytorch
from im2mesh import config
from im2mesh.checkpoints import CheckpointIO
from im2mesh.common import make_3d_grid

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device = torch.device('cuda')
cfg = config.load_config('configs/pointcloud/onet_pretrained.yaml', 'configs/default.yaml')

dataset = config.get_dataset('test', cfg, return_idx=True)
model = config.get_model(cfg, device=device, dataset=dataset)

checkpoint_io = CheckpointIO(cfg['training']['out_dir'], model=model)
re = checkpoint_io.load(cfg['test']['model_file'])

https://s3.eu-central-1.amazonaws.com/avg-projects/occupancy_networks/models/onet_pcl2mesh-5c0be168.pt
=> Loading checkpoint from url...


In [3]:
def get_data_sample(idx=500):

    data = dataset[idx]
    print(dataset.metadata[dataset.get_model_dict(idx)['category']])
    
    inputs = torch.from_numpy(data.get('inputs'))
    inputs = inputs.unsqueeze(0).to(device)

    points = torch.from_numpy(data.get('points'))
    points = points.unsqueeze(0).to(device)
    return inputs, points

def predict(inputs, points):
    kwargs = {}
    model.eval()
    with torch.no_grad():
        c = model.encode_inputs(inputs)
        occ_hat = model.decode(points, z=None, c=c, **kwargs).probs

    return occ_hat.squeeze()

In [57]:
nx = 32
box_size = 1.1
points_grid = box_size * make_3d_grid((-0.5,)*3, (0.5,)*3, (nx,)*3)
points_grid = points_grid.unsqueeze(0).to(device)

In [58]:
inputs, points = get_data_sample(idx=2000)
occ_hat = predict(inputs, points_grid)

{'id': '02691156', 'name': 'airplane,aeroplane,plane', 'idx': 4}


In [59]:
occ_hat_pred = torch.where(occ_hat > 0.5)
print('num of points eval: {}'.format(occ_hat.shape))
print('num of pred points: {}'.format(occ_hat_pred[0].shape))

num of points eval: torch.Size([32768])
num of pred points: torch.Size([175])


In [60]:
points_grid_np = points_grid.squeeze().cpu()
grid_colors = np.zeros(points_grid_np.shape)
grid_colors[:, 2] = 1
grid_pc = o3d.geometry.PointCloud()
grid_pc.points = o3d.utility.Vector3dVector(points_grid_np)
grid_pc.colors = o3d.utility.Vector3dVector(grid_colors)

points_grid_pred = points_grid[0, occ_hat_pred[0]].cpu().numpy()
output_colors = np.zeros(points_grid_pred.shape)
output_colors[:, 0] = 1
output_pc = o3d.geometry.PointCloud()
output_pc.points = o3d.utility.Vector3dVector(points_grid_pred)
output_pc.colors = o3d.utility.Vector3dVector(output_colors)


inputs_np = inputs.squeeze().cpu().numpy()
input_colors = np.zeros(inputs_np.shape)
input_colors[:, 1] = 1
input_pc = o3d.geometry.PointCloud()
input_pc.points = o3d.utility.Vector3dVector(inputs_np)
input_pc.colors = o3d.utility.Vector3dVector(input_colors)

o3d.visualization.draw_geometries([output_pc, input_pc])

In [69]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel='rbf', mean_fun='zero'):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        if mean_fun == 'zero':
            self.mean_module = gpytorch.means.ZeroMean()
            
        if kernel == 'rbf':
            self.covar_module = gpytorch.kernels.RBFKernel()
        
        self.set_hparams()
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
    def set_hparams(self, likelihood_noise=0.05, lengthscale=None):
        if likelihood_noise is not None:
            self.likelihood.noise = likelihood_noise
        if lengthscale is not None:
            self.covar_module.lengthscale = lengthscale

def evaluate_gp(model, likelihood, test_x):
    model.eval()
    likelihood.eval()

    with torch.no_grad():
        pred_y = likelihood(model(test_x))
    
    return pred_y

def train_gp(model, likelihood, num_iter=1000):
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.5) 
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(num_iter):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
#         print('Iter %d/%d - Loss: %.3f' % (i + 1, num_epochs, loss.item()))
        optimizer.step()

In [70]:
train_x = inputs.squeeze().cpu()
train_y = torch.ones(train_x.shape[0])
likelihood_rbf = gpytorch.likelihoods.GaussianLikelihood()
model_rbf = ExactGPModel(train_x, train_y, likelihood_rbf, kernel='rbf')

In [71]:
print('rbf likelihood noise:   {}'.format(model_rbf.likelihood.noise.item()))
print('rbf kernel lengthscale: {}'.format(model_rbf.covar_module.lengthscale.item()))
train_gp(model_rbf, likelihood_rbf)
print('rbf likelihood noise:   {}'.format(model_rbf.likelihood.noise.item()))
print('rbf kernel lengthscale: {}'.format(model_rbf.covar_module.lengthscale.item()))

rbf likelihood noise:   0.05000000074505806
rbf kernel lengthscale: 0.6931471824645996
rbf likelihood noise:   0.00010002298949984834
rbf kernel lengthscale: 68.6805648803711


In [73]:
res_gp = 32
points_grid_gp = box_size * make_3d_grid((-0.5,)*3, (0.5,)*3, (res_gp,)*3)
pred_y = evaluate_gp(model_rbf, likelihood_rbf, points_grid_gp)

In [74]:
pred_y.mean.mean()

tensor(1.0000)